# Entendiendo el problema

Como ya sabemos queremos hacer un bot automatico de respuestas cortas para algunas IM. 

Aquí vienen un par preguntas:
   - ¿Que tipo de respuestas dara nuestro bot?
   - ¿Como podemos encarar este problema de la mejor manera posible?

Para poder contestar estas preguntas hay que ver los datos que tenemos hasta ahora.

In [1]:
import pandas as pd
import csv

In [63]:
df_pre1 = pd.read_csv('preprocessed_corpus_pre1.csv')
df_pre2 = pd.read_csv('preprocessed_corpus_pre2.csv')
df_pre3 = pd.read_csv('preprocessed_corpus_pre3.csv')
df_pre4 = pd.read_csv('preprocessed_corpus_pre4.csv')

In [68]:
df = df_pre1, df_pre2, df_pre3, df_pre4

In [64]:
df_pre1.head(10)

,Unnamed: 0,id,owner,date,hour,message
0,0,1,USER0,1/27/16,12:24,MEDIA
1,1,1,USER0,2/4/16,09:50,lee esta historia ! . una mujer muy enferma...
2,2,1,USER0,4/4/16,11:28,hola dani ! vos sabes a donde venden fotocopia...
3,3,1,USER1,4/4/16,11:37,le pregunto a mi jefe
4,4,1,USER0,4/4/16,11:38,gracias ! para cuando sabes ?
5,5,1,USER0,4/4/16,11:38,💃
6,6,1,USER1,4/4/16,11:40,le pregunto ahora a ver si sabe algo : p
7,7,1,USER0,4/4/16,11:59,👏
8,8,1,USER0,4/6/16,08:46,"✳ whatsapp le comunica , . que debido a la..."
9,9,1,USER1,4/6/16,08:46,ja


In [65]:
distribution_pre1 = pd.read_csv('distribution_pre1.csv')
distribution_pre2 = pd.read_csv('distribution_pre2.csv')
distribution_pre3 = pd.read_csv('distribution_pre3.csv')
distribution_pre4 = pd.read_csv('distribution_pre4.csv')

In [66]:
distribution = distribution_pre1, distribution_pre2, distribution_pre3, distribution_pre4

Uno pensaría que los datos no son de mucha utilidad, sin embargo después de discutir un largo rato decidimos encarar el problema como si fuera un problema de machine learning supervisado donde el contexto de los mensajes serían los datos entrenamiento y como etiquetas las respuestas cortas.

Perfecto! Ya sabemos como encararlo... pero antes de empezar debemos aclarar un par definiciones:

   - ¿que es una respuesta corta? cualquier turno (de diálogo) hasta 3 palabras que ocurren por lo menos n veces en el corpus. Haremos varios experimentos con diferentes longitud de respuestas cortas.
   - ¿que es el contexto de una respuesta corta? El contexto de una respuesta corta son los mensajes que ocurren t turnos anteriores a una respuesta corta.

## Distribución de respuestas cortas

Veamos ahora la distribución de las respuestas cortas

In [70]:
from collections import Counter

N = 5
d = {}

counter_pre1 = Counter([' '.join(str(answer).split()) for answer in df_pre1.message if 0 < len(str(answer).split()) <= N])
counter_pre2 = Counter([' '.join(str(answer).split()) for answer in df_pre2.message if 0 < len(str(answer).split()) <= N])
counter_pre3 = Counter([' '.join(str(answer).split()) for answer in df_pre3.message if 0 < len(str(answer).split()) <= N])
counter_pre4 = Counter([' '.join(str(answer).split()) for answer in df_pre4.message if 0 < len(str(answer).split()) <= N])

In [87]:
distribution_answer_pre1 = pd.Series(counter_pre1)
distribution_answer_pre2 = pd.Series(counter_pre2)
distribution_answer_pre3 = pd.Series(counter_pre3)
distribution_answer_pre4 = pd.Series(counter_pre4)

In [61]:
distribution_answer_pre1.sort_values('counter', ascending=False).head(10)

,answer,counter
0,MEDIA,16066
4,ja,8857
47,ok,1197
85,😂,1191
55,si,965
38,NUM,556
303,?,525
3358,🤣,469
159,no,446
337,dale,433


In [89]:
distribution_answer_pre1.to_csv(path='distribution_answer_pre1.csv')
distribution_answer_pre2.to_csv(path='distribution_answer_pre2.csv')
distribution_answer_pre3.to_csv(path='distribution_answer_pre3.csv')
distribution_answer_pre4.to_csv(path='distribution_answer_pre4.csv')

In [90]:
distribution_answer = distribution_answer_pre1, distribution_answer_pre2, distribution_answer_pre3, distribution_answer_pre4

## Etiquetando los contextos

### Respecto a una respuesta corta

Una vez definido el problema, podemos empezar a etiquetar los contextos.

Al principio diferenciaremos los etiquetados según el tamaño de respuestas: 
   - respuesta cortas hasta 1 a 3 palabras

Empezaremos a etiquetar los contextos sin importar los turnos. Para ello seguiremos un par de suposiciones:
   -  Si hay una respuesta corta:
       - Si esta en el mismo chat:
           - si no es la misma persona que escribe quiere decir que hubo una respuesta corta. Agregamos esta respuesta corta al contexto siguiente.
           - si es la misma persona que escribe la respuesta corta la agregamos al contexto.
       - Si esta en diferente chat empezamos un nuevo contexto debido a que no hubo una respuesta corta después de todo.
   - Si es una respuesta larga:
        - Si esta en el mismo chat se agrega al contexto
        - Si no esta, el empezamos un nuevo contexto

In [96]:
def is_short(answer, answer_length, threshold, pre_n):
    """
    is short answer?
    """
    return (0 < len(answer.split()) <= answer_length) and distribution_answer[pre_n][' '.join(answer.split())] >= threshold

def is_same_owner(own1, own2):
    """
    return whenever own1 is the same as own2
    """
    return own1==own2

def is_same_chat(idc1, idc2):
    """
    Is it the same chat?
    """
    return idc1 == idc2

In [97]:
fieldnames = ['idchat', 'context', 'label']
threshold = 10

def label_the_data(filename, answer_length, pre_n):
    with open(filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writeheader()

        data = df[pre_n]
        last_owner = ''
        last_chat = 0
        row = {'idchat': 0, 'context': '', 'label': ''}
        for id, actual_chat, owner, date, hour, text in data.values:
            # respuesta corta
            if is_short(text, answer_length, threshold, pre_n):
                # es el mismo chat
                if is_same_chat(actual_chat, last_chat):
                    # si no es la misma persona que escribe quiere decir que hubo una respuesta (en este caso corta).
                    # el label será la respuesta en sí. Empezará un contexto nuevo de la otra persona.
                    if not is_same_owner(last_owner, owner):
                        row['label'] = text
                        writer.writerow(row)
                        row['context'] = text
                    else:
                        row['context'] += ' . ' + text
                # diferente chat
                else:
                    row['context'] = text
            # respuesta larga           
            else:
                # es el mismo chat
                if is_same_chat(actual_chat, last_chat):
                    row['context'] += ' . ' + text
                # es diferente chat
                else: 
                    row['context'] = text

            row['idchat'] = actual_chat
            last_owner = owner
            last_chat = actual_chat

In [98]:
label_the_data('short_ans1.csv', 1, 1)
label_the_data('short_ans2.csv', 2, 1)
label_the_data('short_ans3.csv', 3, 1)

### Respecto a los turnos anteriores

Luego etiquetaremos de acuerdo a los turnos, es decir eligiremos el contexto de acuerdo con:
   - el turno inmediatamente anterior de respuesta corta
   - los 3 turnos inmediatamente anteriores de respuesta corta
   - los 10 turnos inmediatamente anteriores de respuesta corta

In [110]:
def label_with_turn(filename, frame, answer_length, pre_n):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, dialect='unix')
        writer.writeheader()

        # No uso el datetime
        # Tampoco tengo en cuenta el usuario que escribe
        row = {'idchat': 0, 'context': '', 'label': ''}
        data = df[pre_n]
        last_chat = 0
        context = []
        for id, actual_chat, owner, data, hour, text in data.values:
            if is_short(text, answer_length, threshold, pre_n):
                if is_same_chat(actual_chat, last_chat):
                    row['label'] = text
                    row['context'] = ' '.join(context)
                    writer.writerow(row)
                # No petenece al mismo chat, reiniciamos el contexto
                else:
                    context = []

            # Descartamos la primer conversación en la lista 
            # y agregamos la última 
            if len(context) == frame:
                context = context[1:]
                context.append(text)
            else:
                context.append(text)
            
            row['idchat'] = actual_chat
            last_chat = actual_chat

In [111]:
label_with_turn('turn1_short1_pre2.csv', 1, 1, 1)
label_with_turn('turn1_short2_pre2.csv', 1, 2, 1)
label_with_turn('turn1_short3_pre2.csv', 1, 3, 1)
label_with_turn('turn3_short1_pre2.csv', 3, 1, 1)
label_with_turn('turn3_short2_pre2.csv', 3, 2, 1)
label_with_turn('turn3_short3_pre2.csv', 3, 3, 1)
label_with_turn('turn10_short1_pre2.csv', 10, 1, 1)
label_with_turn('turn10_short2_pre2.csv', 10, 2, 1)
label_with_turn('turn10_short3_pre2.csv', 10, 3, 1)